In [ ]:
from itertools import chain
from nltk.corpus.reader import ConllCorpusReader

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics


In [ ]:
nltk.download('averaged_perceptron_tagger')

nltk.download('conll2002')

In [ ]:
nltk.corpus.conll2002.fileids()

In [ ]:
#train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
#test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
#train_sents = list(nltk.corpus.conll2002.iob_sents('ned.train'))
#test_sents = list(nltk.corpus.conll2002.iob_sents('ned.testb'))

In [1]:
#Estudar depois como usar o conll2003
with open('/home/82068895153/POS/skweak/data/conll2003_dataset/train.txt', 'r') as file:
  sentences = list(file.readlines())

  print (sentences[5])

rejects VBZ B-VP O



In [ ]:
l1_sentences = []
l1_=[]
for token in sentences[5:]:
    cls = token.split()
    if len(cls) !=0:
        l1_.append(cls)
    else:
        l1_sentences.append(l1_)
        l1_= []

In [65]:
setences = l1_sentences

In [ ]:
def split_remove_2(sent):
    sent = sent.split()
    if len(sent)>3:
        return [sent[0]]+sent[2:4]
    else:
        return []


In [ ]:
sentences = [split_remove_2(sent) for sent in l1_sentences]

In [62]:
def word2features(sent, i):
    word = sent[i][0]
    #print (word)
    postag = sent[i][1]
    #print (postag)

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features


In [63]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]


In [ ]:
#train_sents = [nltk.pos_tag(sent) for sent in l1_sentences]

In [68]:
X_train = [sent2features(s) for s in sentences] 

#X_train = [sent2features(s) for s in train_sents]
# y_train = [sent2labels(s) for s in train_sents]


IndexError: string index out of range

In [67]:
y_train = [sent2labels(s) for s in sentences]

ValueError: not enough values to unpack (expected 3, got 1)

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

In [ ]:
labels = list(crf.classes_)
labels.remove('O')
labels

In [ ]:
with open('/home/82068895153/POS/skweak/data/conll2003_dataset/train_out_2.txt', 'r') as file:
  sentences = list(file.readlines())

  print (sentences[5])

In [ ]:
clean_sentences = [word.split() for word in sentences if word]
print (clean_sentences[5])

In [ ]:
tagged_sentences = [nltk.pos_tag(sent) for sent in clean_sentences]
print (tagged_sentences[5])

In [ ]:
X_test =  [sent2features(s) for s in tagged_sentences]
print (X_test[5])


In [ ]:
#Aqui fiquei na duvida de quem seria o 
y_test = [sent2labels(s) for s in tagged_sentences]

In [ ]:
#labels
y_pred = crf.predict(X_test)
print (y_pred[5]) 

In [ ]:
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

In [ ]:
ner = [list(zip(t[0],t[1])) for t in zip(tagged_sentences, labels)]
ner

In [ ]:
#ner[0:1]
str(ner[0:1])
linha

In [ ]:
texto = ''
for linha in (str(ner[0:1])):
    p = 
    texto=texto + p + '\n'

In [ ]:
with open('/home/82068895153/POS/skweak/data/conll2003_dataset/train_ner.txt', 'wt') as fileout:
  fileout.write(texto)